In [1]:
# TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras

import numpy as np

import os
import gc
import skimage
import random

In [2]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".png")]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(d)
    for i in range(len(images)):
        images[i] = images[i]/255
    c = list(zip(images, labels))
    random.shuffle(c)
    images, labels = zip(*c)
    labels= [int(i) for i in labels]
    
    return np.array(images), np.array(labels)

ROOT_PATH = "C:/Users/Nikita/Desktop/T-Systems"
train_data_directory = os.path.join(ROOT_PATH, "TrainGray")

In [3]:
images, labels = load_data(train_data_directory)

def get_data(t):
    
    images_test = images[t*9000:(t+1)*9000]
    labels_test = labels[t*9000:(t+1)*9000]
    
    #images_test = np.array(images_test)
    #labels_test = np.array(labels_test)
    
    images_train = np.concatenate((images[0:t*9000], images[(t+1)*9000:]))
    labels_train = np.concatenate((labels[0:t*9000], labels[(t+1)*9000:]))
    
    return images_train, labels_train, images_test, labels_test

class_names = ['button', 'checkbox', 'radiobutton', 'slider', 'spinner', 'textfield']

In [ ]:
for i in range(5):
    model = keras.Sequential([
        keras.layers.Conv2D(input_shape=(100, 150, 1), kernel_size = (5, 5), filters = 32, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Conv2D(kernel_size = (5, 5), filters = 64, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Conv2D(kernel_size = (5, 5), filters = 128, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Conv2D(kernel_size = (5, 5), filters = 256, activation='relu'),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        #keras.layers.Conv2D(kernel_size = (3, 3), filters = 512, activation='relu'),
        #keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(6, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
    
    images_train, labels_train, images_test, labels_test = get_data(i)
    
    images_train = images_train.reshape(-1, 100, 150, 1)
    images_test = images_test.reshape(-1, 100, 150, 1)
    
    model.fit(images_train, labels_train, epochs=30)
    
    loss, acc = model.evaluate(images_test, labels_test)
    print("Result: " + str(acc))
    
    del images_train
    del labels_train
    del images_test
    del labels_test
    
    gc.collect()